In [18]:
from flask import Flask
from flask import request
import werkzeug
import cv2
from PIL import Image
import mediapipe as mp
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Class containing Final Results for MediaPipe

In [19]:
class Results:
    
    cols = ["x","y","z","vis"]
    #TRAGULAR DATA
    trag_cols = ["x","y","z","vis"]
    trag_left_rows = ["left_ear","left_index","left_elbow","left_wrist"]
    trag_right_rows = ["right_ear","right_index","right_elbow","right_wrist"]
    tragular_left_coord = []
    tragular_right_coord = []
    
    #LUMBAR DATA
    lumbar_cols = ["x","y","z","vis"]
    lumbar_rows = ["left_index_world","right_index_world","left_index_hol","right_index_hol",
                  "left_wrist","right_wrist","left_elbow","right_elbow"]
    lumbar_neutral_coord = []
    lumbar_left_coord = []
    lumbar_right_coord = []
    
    #INTERMALLEOLAR DATA
    intermalleolar_rows = ["left_ankle_world","right_ankle_world","left_ankle_hol"
                           ,"right_ankle_hol","left_knee_hol","right_knee_hol"]
    intermalleolar_coord = []
    
    #CERVICAL DATA
    cervical_cols = ["x","y","z","vis"]
    cervical_left_coord = []
    cervical_neutral_coord = []
    cervical_right_coord = []
    
    #FLEXION DATA
    flexion_neutral_left_rows = ["left_shoulder_neutral_world","left_shoulder_neutral_norm",
                         "left_hip_neutral_world","left_hip_neutral_norm"]
    flexion_extension_left_rows = ["left_shoulder_extension_world","left_shoulder_extension_norm",
                         "left_hip_extension_world","left_hip_extension_norm"]
    flexion_neutral_right_rows = ["right_shoulder_neutral_world","right_shoulder_neutral_norm",
                         "right_hip_neutral_world","right_hip_neutral_norm"]
    flexion_extension_right_rows = ["right_shoulder_extension_world","right_shoulder_extension_norm",
                         "right_hip_extension_world","right_hip_extension_norm"]
    
    flexion_neutral_left_coord = []
    flexion_neutral_right_coord = []
    flexion_extension_left_coord = []
    flexion_extension_right_coord = []
    
    

In [20]:
def draw(annotated_image,results,mp_holistic):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    
    mp_drawing.draw_landmarks(
        annotated_image, 
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS)
    
    mp_drawing.draw_landmarks(
        annotated_image, 
        results.right_hand_landmarks, 
        mp_holistic.HAND_CONNECTIONS)
    
    mp_drawing.draw_landmarks(
        annotated_image,
        results.face_landmarks,
        mp_holistic.FACEMESH_TESSELATION,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=1, circle_radius=1),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),thickness=1, circle_radius=1))
    
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.
        get_default_pose_landmarks_style())
    

In [21]:
#Function used to calculate relevant lumbar coordinates
def mp_pose_lumbar(side,file):
    
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode = True,model_complexity = 2, enable_segmentation=True,min_detection_confidence = 0.5)
    image = cv2.imread(file)
    results = pose.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=5, circle_radius=6),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
        thickness=5, circle_radius=5))
    mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imwrite("lumbar.png",image)

    coordinate_list = [[results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].x,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].y,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].z,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].visibility],
                           [results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].x,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].y,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].z,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].visibility]]

    
    if(side == -1):
        Results.lumbar_left_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_left_coord.index = Results.lumbar_rows
        print(Results.lumbar_left_coord)
    elif(side == 0):
        Results.lumbar_neutral_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_neutral_coord.index = Results.lumbar_rows
        print(Results.lumbar_neutral_coord)
    else:
        Results.lumbar_right_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_right_coord.index = Results.lumbar_rows
        print(Results.lumbar_right_coord)
    
    

# Functions for Measurements
    

In [22]:
#Function used to calculate the relevant tragular coordinates
def mp_pose_tragular(side,file):
    mp_holistic = mp.solutions.holistic 

    holistic = mp_holistic.Holistic(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        refine_face_landmarks=True)
    
    image = cv2.imread(file)
    results = holistic.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    annotated_image = image.copy()
    
    draw(annotated_image,results,mp_holistic)
    
    cv2.imwrite("holistic_tragular.png",annotated_image)
    
    
    if(side == 0):
        left_hand = results.left_hand_landmarks
        
        left_wrist_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].z,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].visibility]
        
        left_elbow_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].x,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].z,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].visibility]
        
        left_ear_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].z,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].visibility]
        
        left_index_coord = [left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].x,
                                       left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].y,
                                       left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].z,
                                       left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].visibility]
        
        
        
        coordinate_list = [left_ear_coord,left_index_coord,left_elbow_coord,left_wrist_coord]
        
        Results.tragular_left_coord = pd.DataFrame(coordinate_list,columns=Results.trag_cols)
        Results.tragular_left_coord.index = Results.trag_left_rows
        print(Results.tragular_left_coord)
    else:
        right_hand = results.right_hand_landmarks
        
        right_wrist_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].z,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].visibility]
        
        right_elbow_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].z,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].visibility]
        
        right_ear_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].x,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].y,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].z,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].visibility]
        
        right_index_coord = [right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].x,
                                       right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].y,
                                       right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].z,
                                       right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].visibility]
        
        
        coordinate_list = [right_ear_coord,right_index_coord,right_elbow_coord,right_wrist_coord]
        
        Results.tragular_right_coord = pd.DataFrame(coordinate_list,columns=Results.trag_cols)
        Results.tragular_right_coord.index = Results.trag_right_rows
        print(Results.tragular_right_coord)
    

In [23]:
def mp_lumbar_holistic(side, file):
    mp_holistic = mp.solutions.holistic 
    
    holistic = mp_holistic.Holistic(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        refine_face_landmarks=True)
    
    image = cv2.imread(file)
    results = holistic.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    annotated_image = image.copy()
    
    draw(annotated_image,results,mp_holistic)
    
    cv2.imwrite("lumbar.png",annotated_image)
    
    right_hand = results.right_hand_landmarks
    left_hand = results.left_hand_landmarks
    
    left_wrist_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].z,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].visibility]
        
    left_elbow_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].x,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].z,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].visibility]
        
    left_index_coord_hol = [left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].x,
                                    left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].y,
                                    left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].z,
                                    left_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].visibility]
    
    left_index_coord_world = [results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_INDEX].x,
                                    results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_INDEX].y,
                                    results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_INDEX].z,
                                    results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_INDEX].visibility]
        
    right_wrist_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].z,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].visibility]
        
    right_elbow_coord = [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].z,
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].visibility]
        
    right_index_coord_hol = [right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].x,
                                    right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].y,
                                    right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].z,
                                    right_hand.landmark[mp_holistic.HandLandmark.INDEX_FINGER_TIP].visibility]
        
    right_index_coord_world =  [results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].x,
                                    results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].y,
                                    results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].z,
                                    results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].visibility]

    coordinate_list = [left_index_coord_world, right_index_coord_world,  left_index_coord_hol, right_index_coord_hol,
                      left_wrist_coord, right_wrist_coord, left_elbow_coord, right_elbow_coord]

    
    if(side == -1):
        Results.lumbar_left_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_left_coord.index = Results.lumbar_rows
        print(Results.lumbar_left_coord)
        
    elif(side == 0):
        Results.lumbar_neutral_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_neutral_coord.index = Results.lumbar_rows
        print(Results.lumbar_neutral_coord)
        
    else:
        Results.lumbar_right_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_right_coord.index = Results.lumbar_rows
        print(Results.lumbar_right_coord)
    
    

In [24]:
def mp_pose_intermalleolar(file):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode = True,model_complexity = 2, enable_segmentation=True,min_detection_confidence = 0.5)
    image = cv2.imread(file)
    results = pose.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=5, circle_radius=6),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
        thickness=5, circle_radius=5))
    mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imwrite("intermalleolar.png",image)
    
    coordinate_list = [[results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].x,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].y,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].z,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].visibility],
                       
                            [results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].x,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].y,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].z,
                            results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].visibility],
                       
                            [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].x,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].y,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].z,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].visibility],
                       
                            [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].x,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].y,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].z,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].visibility],
                      
                            [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].x,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].y,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].z,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE].visibility],
                       
                            [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].x,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].y,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].z,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE].visibility]]
    
    Results.intermalleolar_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
    Results.intermalleolar_coord.index = Results.intermalleolar_rows
    

In [31]:
def mp_pose_cervical(file,side):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode = True,model_complexity = 2, enable_segmentation=True,min_detection_confidence = 0.5)
    image = cv2.imread(file)
    results = pose.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=50, circle_radius=10),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
        thickness=50, circle_radius=10))
    mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imwrite("cervical.png",image)

    if side == -1:
        coordinate_list = [[results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].z,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].visibility],
                            
                            [results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].x,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].y,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].z,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].visibility],
                            
                           [results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].z,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].visibility],
                           
                           [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].x,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].y,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].z,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].visibility]]
            
        Results.cervical_left_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
        Results.cervical_left_coord.index = ["left_nose_world_coordinates", "right_ear_world_coordinates",
                                            "left_nose_norm_coordinates","right_ear_norm_coordinates"]
        print(Results.cervical_left_coord)
        
            
    
    elif side == 0:
        
        coordinate_list = [[results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].z,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].visibility],
                            
                            [results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].x,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].y,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].z,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].visibility],
                           
                           [results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].x,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].y,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].z,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].visibility],
                            
                           [results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].z,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].visibility],
                           
                           [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].x,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].y,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].z,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].visibility],
                          
                          [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].x,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].y,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].z,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].visibility]]
        
        Results.cervical_neutral_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
        Results.cervical_neutral_coord.index = ["neutral_nose_world_coordinates","right_ear_world_coordinates",
                                               "left_ear_world_coordinates","nuetral_nose_norm_coordinates",
                                                "right_ear_norm_coordinates","left_ear_norm_coordinates"]
        print(Results.cervical_neutral_coord)
    else:
        
        coordinate_list = [[results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].z,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].visibility],
                            
                            [results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].x,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].y,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].z,
                       results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].visibility],
                            
                           [results.pose_world_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].z,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].visibility],
                           
                           [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].x,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].y,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].z,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].visibility]]
    
        Results.cervical_right_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
        Results.cervical_right_coord.index = ["right_nose_world_coordinates", "left_ear_world_coordinates",
                                            "right_nose_norm_coordinates","left_ear_norm_coordinates"]
        print(Results.cervical_right_coord)
        
        
    
    
    
    

In [32]:
def mp_pose_flexion(file,side):
    
    mp_holistic = mp.solutions.holistic 
    
    holistic = mp_holistic.Holistic(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        refine_face_landmarks=True)
    
    image = cv2.imread(file)
    results = holistic.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    annotated_image = image.copy()
    
    draw(annotated_image,results,mp_holistic)
    cv2.imwrite("flexion.png",annotated_image)
    coordinate_list = []
    
    if side == -1 or side == 0:
        coordinate_list = [[results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].x,
                         results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y,
                         results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].z,
                         results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].visibility],
                         
                         [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].x,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].z,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].visibility],
                        
                        [results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].x,
                         results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y,
                         results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].z,
                         results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].visibility],
                        
                        [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].x,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].z,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].visibility]]
        if side == -1:
            Results.flexion_neutral_left_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
            Results.flexion_neutral_left_coord.index = Results.flexion_neutral_left_rows
            print(Results.flexion_neutral_left_coord)
        else:
            Results.flexion_extension_left_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
            Results.flexion_extension_left_coord.index = Results.flexion_extension_left_rows
            print(Results.flexion_extension_left_coord)
        
    else:
        coordinate_list = [[results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x,
                         results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y,
                         results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].z,
                         results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].visibility],
                         
                         [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].z,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].visibility],
                        
                        [results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].x,
                         results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y,
                         results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].z,
                         results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].visibility],
                        
                        [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].x,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].z,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].visibility]]
        
        if side == 1:
            Results.flexion_neutral_right_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
            Results.flexion_neutral_right_coord.index = Results.flexion_neutral_right_rows
            print(Results.flexion_neutral_right_coord)
        else:
            Results.flexion_extension_right_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
            Results.flexion_extension_right_coord.index = Results.flexion_extension_right_rows
            print(Results.flexion_extension_right_coord)
        

# Server Code

In [42]:
app = Flask(__name__)

@app.route("/")
def showHomePage():
    return "This is home page"

@app.route("/debug",methods=["POST"])
def debug():
    text = request.form["sample"]
    print(text)
    return "received"

####POSTING RELEVANT IMAGES TO SERVER###

@app.route("/tragularLeft",methods=["POST"])
def tragularLeft():
    imageFile = request.files['leftTragular']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_tragular(0,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/tragularRight",methods=["POST"])
def tragularRight():
    imageFile = request.files['rightTragular']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_tragular(1,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/lumbarLeft",methods=["POST"])
def lumbarLeft():
    imageFile = request.files['leftLumbar']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_lumbar_holistic(-1,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/lumbarNeutral",methods=["POST"])
def lumbarNeutral():
    imageFile = request.files['neutralLumbar']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_lumbar_holistic(0,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/lumbarRight",methods=["POST"])
def lumbarRight():
    imageFile = request.files['rightLumbar']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_lumbar_holistic(1,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/intermalleolar",methods=["POST"])
def intermalleolar():
    imageFile = request.files['intermalleolar']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_intermalleolar(filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/cervicalLeft",methods=["POST"])
def cervicalLeft():
    imageFile = request.files['leftCervical']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_cervical(filename,-1)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/cervicalNeutral",methods=["POST"])
def cervicalNeutral():
    imageFile = request.files['neutralCervical']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_cervical(filename,0)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/cervicalRight",methods=["POST"])
def cervicalRight():
    imageFile = request.files['rightCervical']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_cervical(filename,1)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/flexionLeftNeutral",methods=["POST"])
def flexionLeftNeutral():
    imageFile = request.files['leftFlexionNeutral']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_flexion(filename,-1)
    print("\nSuccessful upload")
    return "Image uploaded successfully"


@app.route("/flexionLeftExtension",methods=["POST"])
def flexionLeftExtension():
    imageFile = request.files['leftFlexionExtension']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_flexion(filename,0)
    print("\nSuccessful upload")
    return "Image uploaded successfully"


@app.route("/flexionRightNeutral",methods=["POST"])
def flexionRightNeutral():
    imageFile = request.files['rightFlexionNeutral']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_flexion(filename,1)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/flexionRightExtension",methods=["POST"])
def flexionRightExtension():
    imageFile = request.files['rightFlexionExtension']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_flexion(filename,2)
    print("\nSuccessful upload")
    return "Image uploaded successfully"
    
    

if __name__ == "__main__":
    app.run(host="0.0.0.0")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://138.38.187.62:5000/ (Press CTRL+C to quit)


In [34]:
Results.cervical_left_coord

[]

In [35]:
Results.intermalleolar_coord

[]

In [36]:
Results.cervical_left_coord

[]

In [37]:
Results.tragular_left_coord

[]